In [1]:
import os
import json
import numpy as np
import cv2
from labelme.utils import shape_to_mask

In [2]:
LABEL_COLORS = {
    "pawn": (255, 0, 0),        # Đỏ
    "knight": (0, 255, 0),      # Xanh lá
    "bishop": (0, 0, 255),      # Xanh dương
    "rook": (255, 255, 0),      # Vàng
    "queen": (255, 0, 255),     # Hồng
    "king": (0, 255, 255)       # Xanh lục bảo
}

In [3]:
def json_to_colored_mask(json_path, output_dir, img_size=(512, 512)):
    with open(json_path, "r") as f:
        data = json.load(f)

    img_path = os.path.join(os.path.dirname(json_path), data["imagePath"])
    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    # Tạo mask ảnh màu với 3 kênh (mặc định là đen)
    mask = np.zeros((h, w, 3), dtype=np.uint8)

    for shape in data["shapes"]:
        label = shape["label"]
        if label in LABEL_COLORS:
            color = LABEL_COLORS[label]  # Lấy màu RGB của nhãn
            points = np.array(shape["points"], dtype=np.int32)

            color = LABEL_COLORS[label][::-1]

            # Điền màu vào mask
            cv2.fillPoly(mask, [points], color=color)

    # Resize mask về kích thước chuẩn
    # mask_resized = cv2.resize(mask, img_size, interpolation=cv2.INTER_NEAREST)

    # Lưu mask màu thành ảnh PNG
    mask_path = os.path.join(output_dir, os.path.basename(json_path).replace(".json", ".png"))
    cv2.imwrite(mask_path, mask)


In [9]:
input_dir = "./ChessPieces_Dataset/train/Knight"
output_dir = "./mask/Knight"
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if file.endswith(".json"):
        json_to_colored_mask(os.path.join(input_dir, file), output_dir)

In [7]:
import os
import json

# Thư mục chứa các file JSON
json_folder = "./ChessPieces_Dataset/train/Pawn"  # Thay bằng đường dẫn thư mục thực tế

# Đếm số lượng file đã cập nhật
updated_count = 0

# Duyệt qua tất cả các file trong thư mục
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):  # Chỉ xử lý file JSON
        file_path = os.path.join(json_folder, filename)

        # Đọc nội dung file JSON
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Kiểm tra và cập nhật "imagePath"
        if "imagePath" in data:
            original_path = data["imagePath"]
            new_path = os.path.basename(original_path)  # Lấy phần tên file

            if new_path != original_path:  # Kiểm tra xem có cần cập nhật không
                data["imagePath"] = new_path
                updated_count += 1

                # Ghi lại file JSON đã chỉnh sửa
                with open(file_path, "w", encoding="utf-8") as file:
                    json.dump(data, file, indent=4, ensure_ascii=False)

# In ra số lượng file đã cập nhật
print(f"Số lượng file JSON đã cập nhật: {updated_count}")


Số lượng file JSON đã cập nhật: 360


In [21]:
import os
import shutil

# Đường dẫn đến thư mục dataset ban đầu
dataset_dir = "./ChessPieces_Dataset/train/Bishop"  # Thư mục chứa 2520 ảnh gốc
output_dir = "./images/Bishop"  # Thư mục mới chỉ chứa ảnh

# Tạo thư mục nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)

# Duyệt qua tất cả file JSON trong dataset gốc
for file in os.listdir(dataset_dir):
    if file.endswith(".json"):  # Nếu file là annotation JSON
        image_path = os.path.join(dataset_dir, file.replace(".json", ".jpg"))  # Giả sử ảnh có đuôi .jpg
        
        # Kiểm tra nếu ảnh tồn tại
        if os.path.exists(image_path):
            # Copy ảnh sang thư mục mới
            shutil.copy(image_path, os.path.join(output_dir, file.replace(".json", ".jpg")))

print(f"✅ Đã copy {len(os.listdir(output_dir))} ảnh vào {output_dir}")


✅ Đã copy 312 ảnh vào ./images/Bishop


In [23]:
import os
import shutil
import random

# Đường dẫn đến thư mục chứa ảnh và mask
image_dir = "./images"
mask_dir = "./masks"
output_dir = "./dataset_split"

# Tạo thư mục train/val/test cho cả ảnh gốc và mask
splits = ["train", "val", "test"]
for split in splits:
    os.makedirs(os.path.join(output_dir, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, split, "masks"), exist_ok=True)

# Danh sách các lớp (tên thư mục)
classes = sorted(os.listdir(image_dir))  # ["Bishop", "King", "Knight", "Pawn", "Queen", "Rook"]

# Chia dữ liệu cho từng lớp
for class_name in classes:
    class_image_dir = os.path.join(image_dir, class_name)
    class_mask_dir = os.path.join(mask_dir, class_name)

    # Lấy danh sách ảnh có mask tương ứng
    image_files = [f for f in os.listdir(class_image_dir) if f.endswith(".jpg") or f.endswith(".png")]
    image_files = [f for f in image_files if os.path.exists(os.path.join(class_mask_dir, f.replace(".jpg", ".png")))]

    # Xáo trộn danh sách ảnh để tránh bias
    random.shuffle(image_files)

    # Chia theo tỷ lệ 80% train, 10% val, 10% test
    total = len(image_files)
    train_size = int(0.8 * total)
    val_size = int(0.1 * total)
    test_size = total - train_size - val_size

    # Hàm copy ảnh và mask vào tập train/val/test
    def copy_data(files, split):
        for file in files:
            img_src = os.path.join(class_image_dir, file)
            mask_src = os.path.join(class_mask_dir, file.replace(".jpg", ".png"))

            img_dst = os.path.join(output_dir, split, "images", file)
            mask_dst = os.path.join(output_dir, split, "masks", file.replace(".jpg", ".png"))

            shutil.copy(img_src, img_dst)
            shutil.copy(mask_src, mask_dst)

    # Copy ảnh vào các tập train/val/test
    copy_data(image_files[:train_size], "train")
    copy_data(image_files[train_size:train_size + val_size], "val")
    copy_data(image_files[train_size + val_size:], "test")

    print(f"✅ {class_name}: {train_size} train, {val_size} val, {test_size} test")

print("🎯 Hoàn thành chia dữ liệu!")

✅ Bishop: 249 train, 31 val, 32 test
✅ King: 280 train, 35 val, 35 test
✅ Knight: 243 train, 30 val, 31 test
✅ Pawn: 288 train, 36 val, 36 test
✅ Queen: 240 train, 30 val, 30 test
✅ Rook: 240 train, 30 val, 30 test
🎯 Hoàn thành chia dữ liệu!
